# Task
Train multiple machine learning models on the Wine dataset from scikit-learn, perform hyperparameter tuning using GridSearchCV and RandomizedSearchCV, evaluate their performance using accuracy, precision, recall, and F1-score, and select the best-performing model.

## Load data

### Subtask:
Load the Wine dataset from scikit-learn.


**Reasoning**:
Import the `load_wine` function, load the dataset, and print its keys to understand its structure.



In [1]:
from sklearn.datasets import load_wine

wine_data = load_wine()
print(wine_data.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])


## Split data

### Subtask:
Split the dataset into training and testing sets.


**Reasoning**:
Split the data into training and testing sets and print their shapes.



In [2]:
from sklearn.model_selection import train_test_split

X = wine_data.data
y = wine_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (142, 13)
Shape of X_test: (36, 13)
Shape of y_train: (142,)
Shape of y_test: (36,)


## Define models

### Subtask:
Define multiple machine learning models to be evaluated.


**Reasoning**:
Import the necessary model classes and instantiate them with default parameters, then store them in a dictionary.



In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    'LogisticRegression': LogisticRegression(),
    'SVC': SVC(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier()
}

print("Defined models:")
for name, model in models.items():
    print(f"- {name}: {model}")

Defined models:
- LogisticRegression: LogisticRegression()
- SVC: SVC()
- DecisionTreeClassifier: DecisionTreeClassifier()
- KNeighborsClassifier: KNeighborsClassifier()


## Hyperparameter tuning

### Subtask:
Implement hyperparameter tuning using GridSearchCV and RandomizedSearchCV for each model.


**Reasoning**:
Implement hyperparameter tuning for each model using GridSearchCV and RandomizedSearchCV, fit the search objects to the training data, and store the best parameters.



In [4]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np

param_grids = {
    'LogisticRegression': {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear']
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'gamma': [1, 0.1, 0.01],
        'kernel': ['rbf', 'linear']
    },
    'DecisionTreeClassifier': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'KNeighborsClassifier': {
        'n_neighbors': np.arange(1, 21),
        'weights': ['uniform', 'distance']
    }
}

best_params = {}

for name, model in models.items():
    if name == 'LogisticRegression':
        search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy')
    elif name == 'SVC':
         search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy')
    elif name == 'DecisionTreeClassifier':
        search = RandomizedSearchCV(model, param_grids[name], n_iter=20, cv=5, scoring='accuracy', random_state=42)
    elif name == 'KNeighborsClassifier':
         search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy')

    search.fit(X_train, y_train)
    best_params[name] = search.best_params_
    print(f"Best parameters for {name}: {best_params[name]}")


Best parameters for LogisticRegression: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Best parameters for SVC: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best parameters for DecisionTreeClassifier: {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 30}
Best parameters for KNeighborsClassifier: {'n_neighbors': np.int64(16), 'weights': 'distance'}


## Train and evaluate

### Subtask:
Train each model with the tuned hyperparameters and evaluate their performance using accuracy, precision, recall, and F1-score on the test set.


**Reasoning**:
Train each model with the tuned hyperparameters and evaluate their performance using accuracy, precision, recall, and F1-score on the test set.



In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

evaluation_results = {}

for name, model in models.items():
    tuned_model = model.__class__(**best_params[name])
    tuned_model.fit(X_train, y_train)
    y_pred = tuned_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    evaluation_results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"--- {name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

--- LogisticRegression ---
Accuracy: 0.9722
Precision: 0.9741
Recall: 0.9722
F1-score: 0.9722
--- SVC ---
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000
--- DecisionTreeClassifier ---
Accuracy: 0.9444
Precision: 0.9514
Recall: 0.9444
F1-score: 0.9449
--- KNeighborsClassifier ---
Accuracy: 0.7500
Precision: 0.7599
Recall: 0.7500
F1-score: 0.7524


## Analyze results

### Subtask:
Analyze the evaluation results and select the best-performing model.


**Reasoning**:
Analyze the evaluation results to identify the best performing model based on the provided metrics.



In [6]:
import pandas as pd

evaluation_df = pd.DataFrame(evaluation_results)
display(evaluation_df)

best_model_name = None
best_f1_score = -1

for model_name, metrics in evaluation_results.items():
    if metrics['f1_score'] > best_f1_score:
        best_f1_score = metrics['f1_score']
        best_model_name = model_name

print(f"\nThe best performing model based on F1-score is: {best_model_name}")

,LogisticRegression,SVC,DecisionTreeClassifier,KNeighborsClassifier
accuracy,0.972222,1.0,0.944444,0.750000
precision,0.974074,1.0,0.951389,0.759877
recall,0.972222,1.0,0.944444,0.750000
f1_score,0.972187,1.0,0.944856,0.752379



The best performing model based on F1-score is: SVC


from matplotlib import pyplot as plt
evaluation_df['LogisticRegression'].plot(kind='hist', bins=20, title='LogisticRegression')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
evaluation_df['DecisionTreeClassifier'].plot(kind='hist', bins=20, title='DecisionTreeClassifier')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
evaluation_df['KNeighborsClassifier'].plot(kind='hist', bins=20, title='KNeighborsClassifier')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['SVC']
  ys = series['KNeighborsClassifier']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = evaluation_df.sort_values('SVC', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('SVC')
_ = plt.ylabel('KNeighborsClassifier')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['SVC']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'SVC'}, axis=1)
              .sort_values('SVC', ascending=True))
  xs = counted['SVC']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = evaluation_df.sort_values('SVC', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('SVC')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
evaluation_df['LogisticRegression'].plot(kind='line', figsize=(8, 4), title='LogisticRegression')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
evaluation_df['DecisionTreeClassifier'].plot(kind='line', figsize=(8, 4), title='DecisionTreeClassifier')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
evaluation_df['KNeighborsClassifier'].plot(kind='line', figsize=(8, 4), title='KNeighborsClassifier')
plt.gca().spines[['top', 'right']].set_visible(False)

## Summary:

### Data Analysis Key Findings

*   The Wine dataset was successfully loaded and split into training and testing sets (80/20 split).
*   Four different machine learning models were defined for evaluation: Logistic Regression, Support Vector Classifier (SVC), Decision Tree Classifier, and K Neighbors Classifier.
*   Hyperparameter tuning was performed using GridSearchCV for Logistic Regression, SVC, and K Neighbors Classifier, and RandomizedSearchCV for Decision Tree Classifier.
*   After tuning and training, the models were evaluated on the test set using accuracy, precision, recall, and F1-score.
*   The SVC model achieved the highest performance across all metrics on the test set, with an accuracy, precision, recall, and F1-score of 1.0000.
*   The K Neighbors Classifier had the lowest performance among the evaluated models.
*   Based on the F1-score, the best-performing model was identified as SVC.

